Task: File Ingestion and Schema validation


ake any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

Read the file ( Present approach of reading the file )

Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational efficiency

Perform basic validation on data columns : eg: remove special character , white spaces from the col name

As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of read and write file, column name in YAML

Validate number of columns and column name of ingested file with YAML.

Write the file in pipe separated text file (|) in gz format.

Create a summary of the file:

Total number of rows,

total number of columns

file size



In [1]:
!pip install dask[complete] modin[ray] ray pyyaml


In [2]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import ray
import yaml
import os
import time


2025-03-25 10:22:11,584	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
pip install -U ipywidgets


   ---------------------------------------- 0.0/139.8 kB ? eta -:--:--
   -------------------------------------- - 133.1/139.8 kB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 139.8/139.8 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/214.4 kB ? eta -:--:--
   ---------------------------------------- 214.4/214.4 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.3 MB 14.2 MB/s eta 0:00:01
   ---------------------------- ----------- 1.6/2.3 MB 26.4 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 21.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.5.2
    Uninstalling widgetsnbextension-3.5.2:
      Successfully uninstalled widgetsnbextension-3.5.2
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling 

In [1]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import ray
import yaml
import os
import time


Read the Data

In [82]:
# Using pandas to read CSV file
df_pandas = pd.read_csv(r'C:\Users\Subhendu\week6\rate-puf.csv')
print(df_pandas.head())


   BusinessYear StateCode  IssuerId SourceName           ImportDate  \
0          2025        AK     21989       HIOS  2024-08-29 01:02:15   
1          2025        AK     21989       HIOS  2024-08-29 01:02:15   
2          2025        AK     21989       HIOS  2024-08-29 01:02:15   
3          2025        AK     21989       HIOS  2024-08-29 01:02:15   
4          2025        AK     21989       HIOS  2024-08-29 01:02:15   

  RateEffectiveDate RateExpirationDate          PlanId   RatingAreaId Tobacco  \
0        2025-10-01         2025-12-31  21989AK0130001  Rating Area 1     NaN   
1        2025-01-01         2025-03-31  21989AK0130001  Rating Area 1     NaN   
2        2025-04-01         2025-06-30  21989AK0130001  Rating Area 1     NaN   
3        2025-07-01         2025-09-30  21989AK0130001  Rating Area 1     NaN   
4        2025-07-01         2025-09-30  21989AK0130001  Rating Area 2     NaN   

             Age  IndividualRate  IndividualTobaccoRate  Couple  \
0  Family Option   

In [83]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 20 entries, BusinessYear to CoupleAndThreeOrMoreDependents
dtypes: float64(9), int64(2), string(9)

In [85]:
#No. of Rows
len(df.index)

2418441

In [86]:
#No, of Columns
len(df.columns)

20

In [4]:
# Method 1: Pandas
start_time = time.time()
df_pandas = pd.read_csv(r'C:\Users\Subhendu\week6\rate-puf.csv')
end_time = time.time()
time_pandas = end_time - start_time
print("Time taken by Pandas:", time_pandas, "seconds")

Time taken by Pandas: 5.135478258132935 seconds


In [10]:
pip install modin[ray]


Note: you may need to restart the kernel to use updated packages.


In [3]:
import time

In [88]:
import modin.pandas as pd  
import time  

# Modin timing  
start_time = time.time()  
df_modin = pd.read_csv(r'C:\Users\Subhendu\week6\rate-puf.csv')  
modin_time = time.time() - start_time  

print(f"Time taken by Modin: {modin_time:.2f} seconds")  


Time taken by Modin: 4.16 seconds


In [92]:
import ray  # Import Ray
import time  # Import time module for timing

ray.init(ignore_reinit_error=True)  # Initialize Ray

# Ray timing
start_time = time.time()
df_ray = ray.data.read_csv(r'C:\Users\Subhendu\week6\rate-puf.csv')  
ray_time = time.time() - start_time

print(f"Ray Time: {ray_time:.2f} seconds")


2025-03-25 20:55:53,261	INFO worker.py:1684 -- Calling ray.init() again after it has already been called.


Ray Time: 0.01 seconds


In [91]:
import dask.dataframe as dd
import time

# Specify dtypes explicitly
dtype_dict = {'Tobacco': 'object'}  # Define Tobacco as a string column

# Read CSV using Dask
start_time = time.time()
df_dask = dd.read_csv(r'C:\Users\Subhendu\week6\rate-puf.csv', dtype=dtype_dict)
df_dask = df_dask.compute()  # Convert to Pandas DataFrame if needed
end_time = time.time()

print(f"Dask Time: {end_time - start_time:.2f} seconds")


Dask Time: 6.77 seconds


Here Ray is better than Pandas, Modin and Ray, with the least reading time of 0.01 sec

In [17]:
# Perform basic validation on data columns

df = pd.read_csv(r'C:\Users\Subhendu\week6\rate-puf.csv')

# remove special characters from column names
df.columns = df.columns.str.replace('[^a-zA-Z0-9]+', '_')

# remove whitespaces from column names
df.columns = df.columns.str.strip().str.lower()

# check for missing values in each column
missing_values = df.isnull().sum()

# check for duplicates
duplicates = df[df.duplicated()]

In [18]:
# get all columns
all_cols = df.columns.tolist()

# print the columns
print(all_cols)

['businessyear', 'statecode', 'issuerid', 'sourcename', 'importdate', 'rateeffectivedate', 'rateexpirationdate', 'planid', 'ratingareaid', 'tobacco', 'age', 'individualrate', 'individualtobaccorate', 'couple', 'primarysubscriberandonedependent', 'primarysubscriberandtwodependents', 'primarysubscriberandthreeormoredependents', 'coupleandonedependent', 'coupleandtwodependents', 'coupleandthreeormoredependents']


In [27]:
import yaml

with open("config.yaml", "r") as file:
    config_data = yaml.safe_load(file)

print(config_data)  # Print to check structure


{'cells': [{'cell_type': 'code', 'execution_count': None, 'id': 'ea840c94', 'metadata': {}, 'outputs': [], 'source': ['fileType: csv\n', 'fileName: rate-puf\n', 'readDelimiter: ","\n', 'writeDelimiter: "|"\n', 'columns:\n', '  - businessyear\n', '  - statecode\n', '  - issuerid\n', '  - sourcename\n', '  - importdate\n', '  - rateeffectivedate\n', '  - rateexpirationdate\n', '  - planid\n', '  - ratingareaid\n', '  - tobacco\n', '  - age\n', '  - individualrate\n', '  - individualtobaccorate\n', '  - couple\n', '  - primarysubscriberandonedependent\n', '  - primarysubscriberandtwodependents\n', '  - primarysubscriberandthreeormoredependents\n', '  - coupleandonedependent\n', '  - coupleandtwodependents\n', '  - coupleandthreeormoredependents\n']}], 'metadata': {'kernelspec': {'display_name': 'Python 3 (ipykernel)', 'language': 'python', 'name': 'python3'}, 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3}, 'file_extension': '.py', 'mimetype': 'text/x-python', 'name

In [31]:
import yaml

# Open and read the YAML file
with open("config.yaml", "r") as file:
    config_data = yaml.safe_load(file)

# Print the loaded data to verify
print(config_data)


{'file_type': 'csv', 'dataset_name': 'file', 'file_name': 'rate-puf', 'table_name': 'edsurv', 'inbound_delimiter': ',', 'outbound_delimiter': '|', 'skip_leading_rows': 1, 'columns': ['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'ImportDate', 'RateEffectiveDate', 'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age', 'IndividualRate', 'IndividualTobaccoRate', 'Couple', 'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents', 'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent', 'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents']}


In [38]:
file_type = config_data.get("file_type")
file_name = config_data.get("file_name")
read_delimiter = config_data.get("inbound_delimiter")
write_delimiter = config_data.get("outbound_delimiter")
columns = config_data.get("columns")

print(f"File Type: {file_type}")
print(f"File Name: {file_name}")
print(f"Read Delimiter: {read_delimiter}")
print(f"Write Delimiter: {write_delimiter}")
print(f"Columns: {columns}")


File Type: csv
File Name: rate-puf
Read Delimiter: ,
Write Delimiter: |
Columns: ['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'ImportDate', 'RateEffectiveDate', 'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age', 'IndividualRate', 'IndividualTobaccoRate', 'Couple', 'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents', 'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent', 'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents']


In [43]:
import os
print(os.getcwd())


C:\Users\Subhendu


In [50]:
df = pd.read_csv(
    r"C:\Users\Subhendu\week6\rate-puf.csv", 
    delimiter=config_data["inbound_delimiter"],
    skiprows=config_data["skip_leading_rows"]
)


In [52]:
import dask.dataframe as dd

# Specify the dtype for the "Tobacco" column to be an object (string)
df_sample = dd.read_csv(
    r'C:\Users\Subhendu\week6\rate-puf.csv', 
    delimiter=',', 
    dtype={'Tobacco': 'object'}  # Explicitly set the dtype for the "Tobacco" column
)

# Display the first few rows
df_sample.head()


,BusinessYear,StateCode,IssuerId,SourceName,ImportDate,RateEffectiveDate,RateExpirationDate,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,IndividualTobaccoRate,Couple,PrimarySubscriberAndOneDependent,PrimarySubscriberAndTwoDependents,PrimarySubscriberAndThreeOrMoreDependents,CoupleAndOneDependent,CoupleAndTwoDependents,CoupleAndThreeOrMoreDependents
0,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-10-01,2025-12-31,21989AK0130001,Rating Area 1,<NA>,Family Option,71.83,NaN,149.41,172.40,172.40,172.40,257.17,257.17,257.17
1,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-01-01,2025-03-31,21989AK0130001,Rating Area 1,<NA>,Family Option,69.24,NaN,144.02,166.17,166.17,166.17,247.87,247.87,247.87
2,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-04-01,2025-06-30,21989AK0130001,Rating Area 1,<NA>,Family Option,70.10,NaN,145.81,168.25,168.25,168.25,250.97,250.97,250.97
3,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-07-01,2025-09-30,21989AK0130001,Rating Area 1,<NA>,Family Option,70.97,NaN,147.62,170.33,170.33,170.33,254.07,254.07,254.07
4,2025,AK,21989,HIOS,2024-08-29 01:02:15,2025-07-01,2025-09-30,21989AK0130001,Rating Area 2,<NA>,Family Option,67.42,NaN,140.23,161.81,161.81,161.81,241.36,241.36,241.36


In [55]:
def col_header_val(df, config_data):
    # Extract the column names from config_data
    expected_columns = config_data.get("columns", [])
    
    # Extract the column names from the dataframe
    actual_columns = df.columns.tolist()
    
    # Check if the actual columns match the expected columns
    if actual_columns == expected_columns:
        print("The header is valid.")
    else:
        print("The header is invalid.")
        print(f"Expected columns: {expected_columns}")
        print(f"Actual columns: {actual_columns}")


In [57]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['2025', 'AK', '21989', 'HIOS', '2024-08-29 01:02:15', '2025-10-01',
       '2025-12-31', '21989AK0130001', 'Rating Area 1', 'Unnamed: 9',
       'Family Option', '71.83', 'Unnamed: 12', '149.41', '172.4', '172.4.1',
       '172.4.2', '257.17', '257.17.1', '257.17.2'],
      dtype='object')
columns of YAML are: ['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'ImportDate', 'RateEffectiveDate', 'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age', 'IndividualRate', 'IndividualTobaccoRate', 'Couple', 'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents', 'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent', 'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents']


In [61]:
# Validate the header of the DataFrame directly without 'util'
if col_header_val(df, config_data) == 0:
    print("Validation failed")
else:
    print("Column validation passed")


The header is invalid.
Expected columns: ['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'ImportDate', 'RateEffectiveDate', 'RateExpirationDate', 'PlanId', 'RatingAreaId', 'Tobacco', 'Age', 'IndividualRate', 'IndividualTobaccoRate', 'Couple', 'PrimarySubscriberAndOneDependent', 'PrimarySubscriberAndTwoDependents', 'PrimarySubscriberAndThreeOrMoreDependents', 'CoupleAndOneDependent', 'CoupleAndTwoDependents', 'CoupleAndThreeOrMoreDependents']
Actual columns: ['2025', 'AK', '21989', 'HIOS', '2024-08-29 01:02:15', '2025-10-01', '2025-12-31', '21989AK0130001', 'Rating Area 1', 'Unnamed: 9', 'Family Option', '71.83', 'Unnamed: 12', '149.41', '172.4', '172.4.1', '172.4.2', '257.17', '257.17.1', '257.17.2']
Validation failed


In [64]:
pip install dask


Note: you may need to restart the kernel to use updated packages.


In [67]:
import datetime
import csv
from dask import dataframe as dd

# Specify the dtype for the 'Tobacco' column
df = dd.read_csv(r'C:\Users\Subhendu\week6\rate-puf.csv', 
                 delimiter=',', 
                 dtype={'Tobacco': 'object'})  # Specify 'Tobacco' as an object type

# Write the DataFrame to a compressed .gz CSV file with pipe separators
df.to_csv('rate-puf.csv.gz',  # Specify the output path and filename
          sep='|',              # Use pipe ('|') as the separator
          header=True,          # Include the header row in the output
          index=False,          # Don't write row indices
          quoting=csv.QUOTE_ALL,  # Quote all fields
          compression='gzip',    # Apply gzip compression
          quotechar='"',         # Character to quote fields
          doublequote=True)      # Allow double quotes within quoted fields


['C:\\Users\\Subhendu\\rate-puf.csv.gz\\0.part',
 'C:\\Users\\Subhendu\\rate-puf.csv.gz\\1.part',
 'C:\\Users\\Subhendu\\rate-puf.csv.gz\\2.part',
 'C:\\Users\\Subhendu\\rate-puf.csv.gz\\3.part',
 'C:\\Users\\Subhendu\\rate-puf.csv.gz\\4.part']

In [74]:
import dask.dataframe as dd
import csv

# Read the CSV with explicit dtype for the 'Tobacco' column
df = dd.read_csv(r'C:\Users\Subhendu\week6\rate-puf.csv', delimiter=',', dtype={'Tobacco': 'object'})

# Save as gzipped CSV with pipe separator
df.to_csv(r'C:\Users\Subhendu\week6\rate-puf.csv.gz', 
          sep='|',              # Using pipe as separator
          header=True,          # Include header
          index=False,          # Exclude index column
          compression='gzip',    # Apply gzip compression
          quoting=csv.QUOTE_ALL) # Quote all fields


['C:\\Users\\Subhendu\\week6\\rate-puf.csv.gz\\0.part',
 'C:\\Users\\Subhendu\\week6\\rate-puf.csv.gz\\1.part',
 'C:\\Users\\Subhendu\\week6\\rate-puf.csv.gz\\2.part',
 'C:\\Users\\Subhendu\\week6\\rate-puf.csv.gz\\3.part',
 'C:\\Users\\Subhendu\\week6\\rate-puf.csv.gz\\4.part']

In [75]:
import gzip
import shutil

# List of .part files
part_files = [
    r'C:\Users\Subhendu\week6\rate-puf.csv.gz\0.part',
    r'C:\Users\Subhendu\week6\rate-puf.csv.gz\1.part',
    r'C:\Users\Subhendu\week6\rate-puf.csv.gz\2.part',
    r'C:\Users\Subhendu\week6\rate-puf.csv.gz\3.part',
    r'C:\Users\Subhendu\week6\rate-puf.csv.gz\4.part'
]

# Output path for the final combined file
output_file = r'C:\Users\Subhendu\week6\rate-puf-combined.csv.gz'

# Combine the .part files into one .gz file
with gzip.open(output_file, 'wb') as f_out:
    for part_file in part_files:
        with open(part_file, 'rb') as f_in:
            shutil.copyfileobj(f_in, f_out)

print("Files combined successfully.")


Files combined successfully.


In [76]:
#number of files in gz format folder
import os
entries = os.listdir(r'C:\Users\Subhendu\week6\rate-puf.csv.gz/')
for entry in entries:
    print(entry)

0.part
1.part
2.part
3.part
4.part


In [79]:
#size of the gz format folder
os.path.getsize(r'C:\Users\Subhendu\week6\rate-puf.csv.gz')

4096